# Upload mangrove extent to GCS and as an asst to GEE

This notebook gets mangrove extent data, uploads to GCS, and creates an earth engine image collection asset.  
These are the steps:  
* We first need to create a single tiff per year of mangrove extent data.
* We then upload the tiffs to GCS.
* We then create an earthengine image collection asset.
* We then upload each image as an asset to GEE.
* Finally make the data also publicly available in Zenodo.

## Setup

In [100]:
import os
from pathlib import Path
import urllib.parse
import geemap
import ee
import zipfile

%run utils.ipynb

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


### Set Cloud credentials (could be done through and env file)

In [4]:
# WARNING: Don't forget to auth to google cloud platform
# gcloud auth application-default login --no-launch-browser --project=mangrove-atlas-246414

### Set Variables for GCS and GEE

In [122]:
#  FIXME: This will depends from where the notebook kernel is running so be careful
WORK_DIR =Path(os.getcwd())
BASE_DIR = f'{WORK_DIR.parents[3]}/datasets'

# @TODO: Add expected data files source as an environment variable.
assert BASE_DIR == '/home/jovyan/work/datasets', f'{BASE_DIR} is not the correct directory'

# variables
data_version = 'v3'
dataset = 'mangrove_aboveground_biomass'

# Set the Google Cloud params
gc_project_id = "mangrove-atlas-246414"
gcs_prefix = "gs://mangrove_atlas"
gcs_bucket = f'mangrove_atlas'

# raw data download
cgs_parent_folder = 'GMW v3.14 geotiffs'
gcs_http_prefix = f"https://storage.googleapis.com/{gcs_bucket}"
base_source_data_url = f'{gcs_http_prefix}/GMW%20v3.14%20geotiffs'

raw_local_folder = Path(f'{BASE_DIR}/raw/{dataset}')
raw_local_folder.mkdir(parents=True, exist_ok=True)

# Image Collection Information
data_year_range = [1996, 2007, 2008, 2009, 2010, 2015, 2016, 2017, 2018, 2019, 2020]
ee_image_collection = f'projects/global-mangrove-watch/mangrove-properties/{dataset}-{data_version}'

no_data_values = [0]
pyramiding = 'MEAN'


In [123]:
description = f"""
## Methodology

This dataset characterizes the global distribution, biomass, and canopy height of mangrove-forested wetlands based on remotely sensed and in situ field measurement data. 
Estimates of (1) mangrove aboveground biomass (AGB), for the nominal year 2000 were derived across a 30-meter resolution global mangrove ecotype extent map using remotely-sensed 
canopy height measurements and region-specific allometric models. Also provided are (4) in situ field measurement data for selected sites across a wide variety of forest 
structures (e.g., scrub, fringe, riverine and basin) in mangrove ecotypes of the global equatorial region. Within designated plots, selected trees were identified to 
species and diameter at breast height (DBH) and tree height was measured using a laser rangefinder or clinometer. Tree density (the number of stems) can be estimated for 
each plot and expressed per unit area. These data were used to derive plot-level allometry among AGB, basal area weighted height (Hba), and maximum canopy height (Hmax) 
and to validate the remotely sensed estimates.  

Spatially explicit maps of mangrove canopy height and AGB derived from space-borne remote sensing data and in situ measurements can be used to assess local-scale 
geophysical and environmental conditions that may regulate forest structure and carbon cycle dynamics. Maps revealed a wide range of canopy heights, including maximum 
values (> 62 m) that surpass maximum heights of other forest types.

### {', '.join(list(map(str, data_year_range[:-1])))} and {data_year_range[-1]} maps of mangrove aboveground biomass (AGB)

For the full documentation, please see the source [methodology](https://daac.ornl.gov/CMS/guides/CMS_Global_Map_Mangrove_Canopy.html).
"""

In [124]:
extent_collection_properties = ImageCollectionProperties(
    name = 'Mangrove aboveground biomass (AGB)',
    version = data_version,
    creator = "Global Mangrove Watch (GMW): Aberystwyth University/soloEO/Wetlands International/UNEP-WCMC/JAXA/DOB Ecology",
    description = description,
    identifier = "",
    keywords = "Erosion; Coasts; Natural Infrastructure; Biodiversity; Blue Carbon; Forests; Mangroves; Landcover",
    citation = "Simard, M., T. Fatoyinbo, C. Smetanka, V.H. Rivera-monroy, E. Castaneda, N. Thomas, and T. Van der stocken. 2019. Global Mangrove Distribution, Aboveground Biomass, and Canopy Height. ORNL DAAC, Oak Ridge, Tennessee, USA. https://doi.org/10.3334/ORNLDAAC/1665",
    license = "https://creativecommons.org/licenses/by/4.0/",
    url = "https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=1665",
    language = "en", 
    altName = "Mangrove aboveground biomass (AGB), Version 3.0",
    distribution = "",
    variableMeasured = "Mangrove aboveground biomass (AGB)",
    units = "1",
    spatialCoverage = "Global tropics",
    temporalCoverage = ','.join(map(str, data_year_range)),
    dataLineage = "Raster data supplied as tilesets per year, each tilset was combined, and added to Google earth engine as multi-temporal ImageCollection."
)

In [125]:
def generate_manifests(year):
    files = [blob.name for blob in list_gcs(bucket_name = gcs_bucket, 
                    dir_path = f'ee_import_data/gmw_{data_version}_{year}', file_pattern = '*.tif') if blob.name.endswith('.tif')]
    manifest = GEEManifest(
            path = Path(f'{BASE_DIR}/processed/manifest/gmw_{data_version}_{year}'),
            name = f"projects/earthengine-legacy/assets/{ee_image_collection}/gmw_{data_version}_{year}",
            tilesets = [ Tilesets(
                sources = [Sources(uris = [f"{gcs_prefix}/{file}"]) for file in files],
                    )
                ],
            start_time = f'{year}-01-01T00:00:00Z',
            end_time = f'{year}-12-31T00:00:00Z',
            uri_prefix = f"",
            properties = ImageProperties(
                band_nodata_values = no_data_values[0],
                band_pyramiding_policies = pyramiding,
                band_names = 'extent',
                year = year,
            ),
            pyramiding_policy = pyramiding,
            missing_data = {'values': no_data_values}

        )
    return manifest

In [126]:
def extract_path_from_zip(path_zip: Path):
    # Extract all the contents of zip file in current directory
    for item in path_zip.iterdir():
        if item.suffix == '.zip':
            with zipfile.ZipFile(item.as_posix(), 'r') as zipObj:
                zipObj.extractall(path_zip)
            
            item.unlink()

In [127]:
def rm_tree(pth: Path):
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()

In [101]:
files = list(list_gcs(bucket_name = gcs_bucket, dir_path = cgs_parent_folder, file_pattern = '*.zip'))

INFO:root:Searching GMW v3.14 geotiffs/*.zip


[<Blob: mangrove_atlas, GMW v3.14 geotiffs/, 1658502357947930>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_1996_gtiff.zip, 1658502502870899>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2007_gtiff.zip, 1658502517196761>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2008_gtiff.zip, 1658502557420942>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2009_gtiff.zip, 1658502628130467>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2010_gtiff.zip, 1658502634901691>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2015_gtiff.zip, 1658502665713112>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2016_gtiff.zip, 1658502699001779>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2017_gtiff.zip, 1658502728032109>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2018_gtiff.zip, 1658502769412890>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2019_gtiff.zip, 1658502783087871>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2020_gtiff.zip, 1658502810948153>]

### Get the raw files locally

In [37]:
copy_folder_gcs(f'{gcs_prefix}/{cgs_parent_folder}/*.zip', raw_local_folder)

Copying gs://mangrove_atlas/GMW v3.14 geotiffs/gmw_v3_1996_gtiff.zip...
Resuming download for /home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_1996_gtiff.zip
Copying gs://mangrove_atlas/GMW v3.14 geotiffs/gmw_v3_2008_gtiff.zip...
Copying gs://mangrove_atlas/GMW v3.14 geotiffs/gmw_v3_2007_gtiff.zip...
Resuming download for /home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2008_gtiff.zip
Copying gs://mangrove_atlas/GMW v3.14 geotiffs/gmw_v3_2009_gtiff.zip...
Resuming download for /home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2007_gtiff.zip
Resuming download for /home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2009_gtiff.zip
Copying gs://mangrove_atlas/GMW v3.14 geotiffs/gmw_v3_2010_gtiff.zip...
Resuming download for /home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2010_gtiff.zip
Copying gs://mangrove_atlas/GMW v3.14 geotiffs/gmw_v3_2015_gtiff.zip...
Copying gs://mangrove_atlas/GMW v3.14 geotiffs/gmw_v3_2016_gtiff.zip...
Copying gs://mangrove_atlas/GMW v3.14 geo

### Extract them and reupload to GCS in the ee_import_data folder

In [96]:
extract_path_from_zip(raw_local_folder)

folder_unziped = [x for x in raw_local_folder.iterdir() if x.is_dir()]

for folder in folder_unziped:
    copy_folder_gcs(folder, f'{gcs_prefix}/ee_import_data/')

[PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2008'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2007'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2009'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2015'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2016'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2017'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_1996'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2019'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2010'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2020'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2018')]

#### Clean up the raw data locally as this is expensive in terms of space.

In [121]:
rm_tree(raw_local_folder)

#### Generate manifests for each year that represent an image in the image collection

In [128]:
list_of_manifests = [generate_manifests(year) for year in data_year_range]

INFO:root:Searching ee_import_data/gmw_v3_1996/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2007/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2008/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2009/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2010/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2015/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2016/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2017/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2018/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2019/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2020/*.tif


#### Create the image collection asset and upload the images to GEE

In [129]:
createImageCollection(ee_asset_path = ee_image_collection,
                        properties = extent_collection_properties, 
                        image_list = list_of_manifests)

INFO:root:Created image collection projects/global-mangrove-watch/land-cover/mangrove_extent-v3
INFO:root:Task created
INFO:root:Upload image projects/earthengine-legacy/assets/projects/global-mangrove-watch/land-cover/mangrove_extent-v3/gmw_v3_1996


Started upload task with ID: SCA6QNE3CPJNNK4J3SZ2BYDZ


INFO:root:Task created
INFO:root:Upload image projects/earthengine-legacy/assets/projects/global-mangrove-watch/land-cover/mangrove_extent-v3/gmw_v3_2007


Started upload task with ID: 6BP27MHNFUOHOPPWLBHGUALZ


INFO:root:Task created
INFO:root:Upload image projects/earthengine-legacy/assets/projects/global-mangrove-watch/land-cover/mangrove_extent-v3/gmw_v3_2008


...

Started upload task with ID: PJOVGOMHETHY7TNFDOLAXNTN


INFO:root:Task created
INFO:root:Upload image projects/earthengine-legacy/assets/projects/global-mangrove-watch/land-cover/mangrove_extent-v3/gmw_v3_2019


Started upload task with ID: VZWXQMIUX7XPRL2JPRXTHWLH


INFO:root:Task created
INFO:root:Upload image projects/earthengine-legacy/assets/projects/global-mangrove-watch/land-cover/mangrove_extent-v3/gmw_v3_2020


Started upload task with ID: GTNM4IUMVSORB4NCG5H5LX57


If we need to add new years to the image collection, we can do that by adding new manifests to the image collection asset.

In [108]:
addImagesToCollection(ee_image_collection, list_of_manifests[0:1])

INFO:root:Task created
INFO:root:Upload image projects/earthengine-legacy/assets/projects/global-mangrove-watch/land-cover/mangrove-extent-test/gmw_v3_1996


Started upload task with ID: CJXPIRIMZJL6MGZ2US6RD3CH


In [112]:
# Check task status
!earthengine task list --status RUNNING

CJXPIRIMZJL6MGZ2US6RD3CH  Upload        Ingest image: "projects/earthengine-lega..  RUNNING    ---


In [ ]:
# Check ImageCollection properties
!earthengine asset info {ee_asset_path}